In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn import model_selection
import numpy as np
import re
import pickle
from sklearn.ensemble import RandomForestRegressor
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
from time import sleep
import urllib.request
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm


options = Options() 
options.add_argument('--headless')
options.add_argument('--disable-gpu')             
options.add_argument('--disable-extensions')       
options.add_argument('--proxy-server="direct://"') 
options.add_argument('--proxy-bypass-list=*')      
options.add_argument('--start-maximized')      
driver = Chrome(executable_path="", options=options)

In [15]:
def getRaceInfoList(id):
    url = 'https://race.netkeiba.com/race/shutuba.html?race_id=' + id
    df = pd.read_html(url)[0]
    newCol = []
    for i in df.columns:
        newCol.append(i[0])
    df.columns = newCol

    target_url = url
    sleep(1)
    driver.get(target_url)
    sleep(1)

    table = driver.find_elements(By.XPATH,'//*[@id="page"]/div[3]/div[2]/table/tbody/tr')
    horse_jockey_id = []
    for i in range(1,len(table)+1):
        result_id = []

        # horse_id
        sleep(1)
        horse_id = driver.find_elements(By.XPATH,'//*[@id="page"]/div[3]/div[2]/table/tbody/tr[' + str(i) +']/td[4]')
        horse_id = re.sub( r'\D+', '', horse_id[0].get_attribute('outerHTML').split('/horse')[1].split('title')[0])
        result_id.append(horse_id)

        # jockey_id
        sleep(1)
        jockey_id = driver.find_elements(By.XPATH,'//*[@id="page"]/div[3]/div[2]/table/tbody/tr[' + str(i) +']/td[7]')
        jockey_id = re.sub( r'\D+', '', jockey_id[0].get_attribute('outerHTML').split('/recent')[1])
        result_id.append(jockey_id)

        horse_jockey_id.append(result_id)  

    df = pd.concat([df, pd.DataFrame(horse_jockey_id,columns=['horse_id', 'jockey_id'])], axis=1)

    horse_id_list = df['horse_id']
    target_url = "https://regist.netkeiba.com/account/?pid=login"
    sleep(1)
    driver.get(target_url)
    sleep(1)

    USERNAME = ""
    PASSWORD = "

    username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[1]/input')
    username_input[0].send_keys(USERNAME)
    sleep(1)

    username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[2]/input')
    username_input[0].send_keys(PASSWORD)
    sleep(1)

    username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/div[1]/input')
    username_input[0].click()
    sleep(1)

    deokure_count_detail = {}
    for horse_id in horse_id_list:
        try:
            sleep(1)
            rece = driver.get('https://db.netkeiba.com/horse/' + horse_id)
            sleep(1)
            table = driver.find_elements(By.XPATH,'//*[@id="contents"]/div[4]/div/table')
            html = table[0].get_attribute('outerHTML')

            deokureCount = 0
            for i in range(len(pd.read_html(html)[0])):
                if '出遅れ' == pd.read_html(html)[0].loc[[i]]['備考'][i]:
                    deokureCount = deokureCount + 1

            horse_detail_list = []
            horse_detail = []
            horse_detail.append(deokureCount)
            horse_detail.append(len(pd.read_html(html)[0]))
            horse_detail_list.append(horse_detail)
            deokure_count_detail[horse_id] = pd.DataFrame(horse_detail_list,columns=['deokureNum','raceNum'])
        except Exception:
            continue

    for key in deokure_count_detail:
        deokure_count_detail[key].index = [key] * len(deokure_count_detail[key])
    results = pd.concat([deokure_count_detail[key] for key in deokure_count_detail], sort=False)
    results = results.reset_index()
    df = pd.concat([df, results], axis=1)

    res = urllib.request.urlopen(url)
    soup = BeautifulSoup(res, 'html.parser')
    raceDate = soup.select('.RaceData01')
    raceDate = raceDate[0].text.replace(' ', '')

    data = raceDate.split('/')[1][0]
    result = []
    for i in range(len(df)):
        if(data == "ダ"):
            result.append(0)
        elif(data=="芝"):
            result.append(1)
        else:
            result.append(2)
    df['track'] = result



    data = raceDate.split('/')[1].split('(')[1].split(')')[0]
    result = []
    for i in range(len(df)):
        result.append(data)
    df['rightORleft'] = result


    data = re.sub( r'\D+', '', raceDate.split('/')[1])
    result = []
    for i in range(len(df)):
        result.append(data)
    df['distance'] = result
    df['distance'] = df['distance'].astype(int)




    data = raceDate.split('/')[2].split(':')[1][0]

    result = []
    for i in range(len(df)):
        if(data== "晴"):
            result.append(0)
        elif(data=="曇"):
            result.append(1)
        elif(data=="雨"):
            result.append(2)
        else:
            result.append(3)
    df['whether'] = result


    data = raceDate.split('/')[3].split(':')[1][0]

    result = []
    for i in range(len(df)):
        if(data == "良"):
            result.append(0)
        elif(data =="稍重"):
            result.append(1)
        elif(data =="重"):
            result.append(2)
        else:
            result.append(3)
    df['situation'] = result


    data = re.sub( r'\D+', '', raceDate.split('/')[0].split(':')[0])
    result = []
    for i in range(len(df)):
        result.append(data)
    df['timeHour'] = result
    df['timeHour'] = df['timeHour'].astype(int)
    return df

In [16]:
# 小倉 202310020201
# 阪神 202309010201
# 東京 202305010601

race_id_list = []
for i in range(1 , 13):
    race_id_list.append('2023100202' + str(i).zfill(2))
    race_id_list.append('2023090102' + str(i).zfill(2))
    race_id_list.append('2023050106' + str(i).zfill(2))

In [ ]:
df = pd.DataFrame()
for race_id in tqdm(race_id_list):
    df = pd.concat([df,getRaceInfoList(race_id)])

In [18]:
initial_Value = df.copy()

In [19]:
df['馬体重(増減)'].str[4:].str.replace(')','', regex=True).str[1:]

0      6
1      8
2      8
3     10
4      2
      ..
11      
12     2
13     6
14     2
15     2
Name: 馬体重(増減), Length: 513, dtype: object

In [ ]:
df['性別'] = df['性齢'].str[0]
df['年齢'] = df['性齢'].str[1:3]
df['年齢'] = df['年齢'].astype(int)
df.drop('性齢', axis=1,inplace=True)
result = []
for i  in df['馬体重(増減)'].str[4:].str.replace(')','', regex=True).str[0]:
    if i=='+':
        result.append(1)
    elif i=='-':
        result.append(2)
    else:
        result.append(0)

df['加減'] = result
df['体重増減'] = df['馬体重(増減)'].str[4:].str.replace(')','', regex=True).str[1:]
df.loc[(df.体重増減 == ''), '体重増減'] = '0'
df.loc[(df.体重増減 == '計不'), '体重増減'] = '0'
df['体重増減'] = df['体重増減'].astype(int)
df['馬体重'] = df['馬体重(増減)'].str[0:3]
# df.loc[(df.馬体重 !='計不'), ['馬体重']].astype(float).mean().round().astype(int)
df.loc[(df.馬体重=='計不'), '馬体重'] = '471'
df.loc[(df.馬体重=='--'), '馬体重'] = '471'
df['馬体重'] = df['馬体重'].astype(int)

In [26]:
df.loc[(df.馬体重=='--'), '馬体重'] = '471'
df['馬体重'] = df['馬体重'].astype(int)

In [27]:
df.drop(['印', '馬名','枠','人気', '騎手', 'お気に入り馬', 'Unnamed: 9_level_0', '厩舎', '馬体重(増減)','index'], axis=1, inplace=True)

In [28]:
df['horse_id'] = df['horse_id'].astype(int)
df['jockey_id'] = df['jockey_id'].astype(int)

In [29]:
dummy = df[['rightORleft','性別']]
dummy = pd.get_dummies(dummy)
newDf = pd.concat([df, dummy], axis=1)
newDf.drop(['rightORleft','性別'],axis=1,inplace=True)

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn import model_selection
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor

df1 = pd.read_pickle('')
df2 = pd.read_pickle('')
df3 = pd.read_pickle('')
df4 = pd.read_pickle('')
df = pd.concat([df1,df2,df3],axis=1)
df = df.merge(df4, on='horse_id')

df.drop(['タイム', '着差', '着順', '馬名', 'ﾀｲﾑ指数', '通過', '上り', '調教ﾀｲﾑ', '厩舎ｺﾒﾝﾄ', '賞金(万円)','調教師','枠番','人気','馬主', '騎手', '単勝'], axis=1, inplace=True)

df = df.drop_duplicates(subset=df.columns)

result = []
for i in  df['time']:
    try:
         appendResult = i.split(':')[0]
         appendResult = re.sub( r'\D+', '', appendResult)
         if(appendResult == ''):
            result.append(0)
            continue
         result.append(appendResult)
    except IndexError:
        result.append(0)
        continue
        

df['timeHour'] = result
df.drop('time', axis=1, inplace=True)

result = []
for i in df['situation']:
    if(i == "良"):
        result.append(0)
    elif(i=="稍重"):
        result.append(1)
    elif(i=="重"):
        result.append(2)
    else:
        result.append(3)
df['situation'] = result

result = []
for i in df['whether']:
    if(i == "晴"):
        result.append(0)
    elif(i=="曇"):
        result.append(1)
    elif(i=="雨"):
        result.append(2)
    else:
        result.append(3)
df['whether'] = result

result = []
for i in df['distance']:
    result.append(re.sub( r'\D+', '', i))
df['distance'] = result
df['distance'] = df['distance'].astype(int)

df.loc[(df.rightORleft=='芝'), 'rightORleft'] = np.nan

result = []
for i in df['track']:
    if(i == "ダ"):
        result.append(0)
    elif(i=="芝"):
        result.append(1)
    else:
        result.append(2)
df['track'] = result

df['性別'] = df['性齢'].str[0]
df['年齢'] = df['性齢'].str[1:3]
df['年齢'] = df['年齢'].astype(int)
df.drop('性齢', axis=1,inplace=True)

result = []
for i  in df['馬体重'].str[4:].str.replace(')','', regex=True).str[0]:
    if i=='+':
        result.append(1)
    elif i=='-':
        result.append(2)
    else:
        result.append(0)

df['加減'] = result
df['体重増減'] = df['馬体重'].str[4:].str.replace(')','', regex=True).str[1:]
df.loc[(df.体重増減 == ''), '体重増減'] = '0'
df['体重増減'] = df['体重増減'].astype(int)
df['馬体重'] = df['馬体重'].str[0:3]
# df.loc[(df.馬体重 !='計不'), ['馬体重']].astype(float).mean().round().astype(int)
df.loc[(df.馬体重=='計不'), '馬体重'] = '471'
df['馬体重'] = df['馬体重'].astype(int)


result = []

for i in df['備考']:
    if i=='出遅れ':
        result.append(1)
    else:
        result.append(0)

df['出遅れ'] = result
df.drop('備考', axis=1, inplace=True)

df['timeHour'] = df['timeHour'].astype(int)
dummy = df[['timeHour','性別','rightORleft']]
dummy = pd.get_dummies(dummy)
dummy = pd.concat([dummy, df[['斤量','馬体重', '馬番','馬体重','体重増減','加減','年齢']]], axis=1)

known_tansyo = dummy[dummy.timeHour != 0].values  
unknown_tansyo = dummy[dummy.timeHour == 0].values


X = known_tansyo[:, 1:]  
y = known_tansyo[:, 0]

rfr = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)
rfr.fit(X, y)

predictedTansyo = rfr.predict(unknown_tansyo[:, 1::])
df.loc[(df.timeHour == 0), 'timeHour'] = predictedTansyo.round().astype(int)

df['rightORleft'].fillna("右", inplace=True)
df['horse_id'] = df['horse_id'].astype(int)
df['jockey_id'] = df['jockey_id'].astype(int)
df['deokureNum'] = df['deokureNum'].astype(int)
df['raceNum'] = df['raceNum'].astype(int)

dummy = df[['rightORleft','性別']]
dummy = pd.get_dummies(dummy)
oldDf = pd.concat([df, dummy], axis=1)
oldDf.drop(['rightORleft','性別'],axis=1,inplace=True)

X = oldDf.drop('出遅れ',axis=1)
y = oldDf['出遅れ']

# lgbm_clf = LGBMClassifier(eval_metric='accuracy')
# lgbm_clf.fit(X_train, y_train)
params = {
    'objective': 'binary',
    'subsample': 0.8,
    'reg_lambda': 0.5,
    'reg_alpha': 0.1,
    'num_leaves': 31,
    'n_estimators': 500,
    'min_child_samples': 30,
    'max_depth': 15,
    'learning_rate': 0.01,
    'colsample_bytree': 1,
    'random_state': 42
}

# LightGBMのモデルを定義する
lgbm_clf = lgb.LGBMClassifier(**params)


lgbm_clf.fit(X, y)

LGBMClassifier(colsample_bytree=1, learning_rate=0.01, max_depth=15,
               min_child_samples=30, n_estimators=500, objective='binary',
               random_state=42, reg_alpha=0.1, reg_lambda=0.5, subsample=0.8)

In [40]:
modelColum = oldDf.columns.to_list()
for i in newDf.columns:
    try:
        modelColum.remove(i)
    except Exception:
        print(i)


for i in modelColum:
    if i == '出遅れ':
        continue
    result = []
    for j in range(len(newDf)):
        result.append(0)
    newDf[i] = result

rightORleft_右外
rightORleft_芝


In [51]:
newDf.drop(['rightORleft_右外', 'rightORleft_芝'], axis=1, inplace=True)

In [52]:
from sklearn.metrics import accuracy_score 
y_pred = lgbm_clf.predict_proba(newDf)

In [53]:
result = pd.DataFrame(y_pred, columns=['出遅れない', '出遅れる'])

In [58]:
pd.merge(result, initial_Value[['馬番', '馬名']],left_index=True, right_index=True).sort_values(by="出遅れる", ascending=False)[50:100]

,出遅れない,出遅れる,馬番,馬名
0,0.985431,0.014569,1,シンフォニー
0,0.985431,0.014569,1,ダブルジョーク
0,0.985431,0.014569,1,ジャストザヴァルス
0,0.985431,0.014569,1,シンコッチョウ
0,0.985431,0.014569,1,ボンベール
0,0.985431,0.014569,1,チュウワハート
0,0.985431,0.014569,1,ボニンブルー
0,0.985431,0.014569,1,リバプールタウン
0,0.985431,0.014569,1,カムランベイ
0,0.985431,0.014569,1,スズカサウスソング


In [ ]:
result = pd.merge(result, initial_Value[['馬番', '馬名']],left_index=True, right_index=True).sort_values(by="出遅れる", ascending=False)

In [ ]:
racePlace = str(url.split('=')[1])[4:6]
raceNumber = str(url.split('=')[1])[10:12]

racePlaceName = ''
if(racePlace == '05'):
    print('東京' + ' ' + raceNumber + 'R' + ' 出遅れ予測ランキング🎯')
    racePlaceName = '東京'
elif(racePlace == '09'):
    print('阪神' + ' ' + raceNumber + 'R' + ' 出遅れ予測ランキング🎯')
    racePlaceName = '阪神'
elif(racePlace == '10'):
    print('小倉' + ' ' + raceNumber + 'R' + ' 出遅れ予測ランキング🎯')
    racePlaceName = '小倉'
print('')

count = 0
for i in result.itertuples():
    count = count + 1
    item = ''
    if(count == 1):
        item = chr(129351)
    elif(count == 2):
        item = chr(129352)
    else:
        item = chr(129353)
    print(item + str(count) + '位 : '+str(i.馬番) + ' ' +i.馬名)
    if(count == 3): break

print('')
print('#' + racePlaceName + raceNumber + 'R' + '  #競馬 #出遅れ予測')

小倉 11R 出遅れ予測ランキング🎯

🥇1位 : 1 ヤマトコウセイ
🥈2位 : 16 ハリーバローズ
🥉3位 : 14 シセイヒテン

#小倉11R  #競馬 #出遅れ予測
